In [1]:
import pandas as pd
import random
from sentence_transformers import SentenceTransformer, util
import torch
import timeit

C:\anaconda new\anaconda\envs\finalproject-patternrecognition\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
pd.set_option('display.max_colwidth', None)
df_match_job = pd.read_csv('df_match_job.csv')
df_match_resume = pd.read_csv('df_match_resume.csv')

In [3]:
df_match_job

,Unnamed: 0,Title,FullDescription,Education,Key Requirements
0,0,"80817 Business Objects Report Writer\nName: Title, dtype: object","80817 Looking for someone with the ability to come in and understand Data Schema and start writing reports in either Webi or Deski\nName: FullDescription, dtype: object","Bachelor's degree in Computer Science, Information Technology or related field.","Knowledge of Data Schema, experience with Webi and Deski reporting tools, strong analytical skills."
1,1,"201344 English Teacher based in Brent\nName: Title, dtype: object","201344 English teacher required for a Long Term position in Brent, North West London. The school requires a committed, gifted teacher of English. The post is suitable to both newly qualified and experienced teachers. If you are appointed you will: Teach English to GCSE Brent Enjoy the opportunity to innovate and to be part of a vibrant successful team Be passionate about young peoples learning. Help improve outcomes at all key stages. ASAP Start This is an excellent opportunity for a newly qualified or experienced teacher of English to contribute to the development of the recently reformed English department. A new Head of Department has been in post for **** year and rapidly developed policy and practice. The department has taken on a new life and positive and creative developments and advancements have been made. The English department is stable and the Head of Department is eager to find a teacher who is keen to join a collaborative, positive and enthusiastic team. This is a large innercity comprehensive with a mixed socioeconomic catchment area and an extremely positive and supportive staff. As in any innercity comprehensive the school has some challenges and is eager to appoint a candidate who is committed to both raising standards and joining a team committed to improving the education offered to pupils The school is looking for CVs from candidates with any level of experience; newly qualified experienced teachers alike. It is A Star Teachers' policy, as a matter of courtesy, to respond to all applications within three working days. However, due to the volume of applications, we are sometimes unable to respond to each applicant individually. If we have not contacted you within three working days, your application has been unsuccessful and your details have not been retained. Please apply for any other position that you may feel suitable for in the future. A Star Teachers is a Quality Marked Agency (continuously since ****) and is committed to safeguarding and promoting the welfare of children and young people.\nName: FullDescription, dtype: object",GCSE,"Committed, gifted teacher of English; Passionate about young people's learning; Help improve outcomes at all key stages; Positive and creative developments and advancements have been made; Committed to raising standards."
2,2,"223076 Research & Development Manager Hosiery exp\nName: Title, dtype: object","223076 Research and Development Manager travel to Hull and UK South Coast plus home working Relocation package **** to **** d.o.e Compressions Hosiery knowledge/experience Responsible for carrying out product development activities on compression hosiery projects as part of the companyR D portfolio. Contribute towards the development of consumer preferred products from the feasibility and concept ideation stage through to the scale up, first production and subsequent launch into market. You will be the face of the business as a companys leading expert in the field of compression hosiery and will have the opportunity to play a key role in shaping the global compression hosiery pipeline. The role requires strong organisational skills and the ability to work independently within a multidisciplinary project team. Strong technical and problem solving skills in all areas of hosiery are an essential requirement of the role, as is an ability to take ownership of key aspects of the project. A good network within the industry woul

In [4]:
df_match_resume

,Unnamed: 0,Extracted,Education,Skills & Experience
0,0,"Education: BE from NMAM Institute of Technology, Poorna Prajna PU College Admar and Ganapathi High School Padubidri. \nKey Skills: C programming basics, Core java, SQL/PLSQL, Quick learner and team player, Good communication skills, Art of convincing and Public speaking. \nExperience: 3 months trining at Infosys; Effectiveness Lab Organizational Effectiveness Labs and Immersive Group Workshop conducted by I-Point Consulting Services Private Limited during the Academic Years 2009-10 & 2010-11; Successfully completed ' Feel Employable' Learning And Development Intervention conducted by AIM INSIGHTS Mangalore; 7 Seminars given on Enzymatic Hydrolysis of Complex Carbohydrates to produce Fermentable and Digestible Sugars; 8 Seminars attended on Microbial Polysaccarides.","BE from NMAM Institute of Technology, Poorna Prajna PU College Admar and Ganapathi High School Padubidri.","C programming basics, Core java, SQL/PLSQL, Quick learner and team player, Good communication skills, Art of convincing and Public speaking. \nExperience: 3 months trining at Infosys; Effectiveness Lab Organizational Effectiveness Labs and Immersive Group Workshop conducted by I-Point Consulting Services Private Limited during the Academic Years 2009-10 & 2010-11; Successfully completed ' Feel Employable' Learning And Development Intervention conducted by AIM INSIGHTS Mangalore; 7 Seminars given on Enzymatic Hydrolysis of Complex Carbohydrates to produce Fermentable and Digestible Sugars; 8 Seminars attended on Microbial Polysaccarides."
1,1,"Education: B.E. in E&TC (2015), Diploma (2012), SSC (2009). Key Skills: C, CSS, Database, Dot, Java. Experiences: Fresher; B.E Project - Designer and Tester; Diploma Project - Designer and Tester; Certifications/Licenses - SCTS dot net developer & PSTS software tester; Other Technical Skills - Basic Java, CSS, C#, SQL MVC & Dot Net.","B.E. in E&TC (2015), Diploma (2012), SSC (2009).","C, CSS, Database, Dot, Java. Experiences: Fresher; B.E Project - Designer and Tester; Diploma Project - Designer and Tester; Certifications/Licenses - SCTS dot net developer & PSTS software tester; Other Technical Skills - Basic Java, CSS, C#, SQL MVC & Dot Net."
2,2,"Education: PGDBM from Narsee Monjee Institute of Management Studies (2017-2018) \nKey Skills: Microsoft Azure SQL DW, Azure Data Factory, SQL Server 2016, Power BI dashboard, Rest API; Program Management (2 years), Product Management (2 years), Quality Assurance (10+ years), Business intelligence, Devops; Visual Studio 2012, Microsoft Test Manager 2012, Azure, Test Automation using Coded UI, Unit Test Framework; Scrum Product Owner & Master. \nExperience: 10+ yrs in Quality Assurance with roles such as Architect at Microsoft India for 5 yrs and Sr Consultant for 4 yrs. Managed test teams for Data Warehouse/SQL Server BI projects & .Net App Development Projects. Played role of Test Lead/Manager in all engagements.",PGDBM from Narsee Monjee Institute of Management Studies (2017-2018),"Microsoft Azure SQL DW, Azure Data Factory, SQL Server 2016, Power BI dashboard, Rest API; Program Management (2 years), Product Management (2 years), Quality Assurance (10+ years), Business intelligence, Devops; Visual Studio 2012, Microsoft Test Manager 2012, Azure, Test Automation using Coded UI, Unit Test Framework; Scrum Product Owner & Master. \nExperience: 10+ yrs in Quality Assurance with roles such as Architect at Microsoft India for 5 yrs and Sr Consultant for 4 yrs. Managed test teams for Data Warehouse/SQL Server BI projects & .Net App Development Projects. Played role of Test Lead/Manager in all engagements."
3,3,"Education: Bachelor Of Technology in insurance from SRIKALAHASTEESWARA INSTITUTE OF TECHNOLOGY \nKey Skills: file systems, Linux, maintenance, scheduling \nExperience: 2.1 years of Experience in IT; Currently working as a Linux Administration at Infosys; Knowledge in installing and maintenance and troubleshooting Linux

In [24]:
def similarity_score(sentence_model, df_match_resume, df_match_job):
    # sentence transformer
    model = SentenceTransformer(sentence_model)
    resume_text_education = str(df_match_resume.iloc[[2]]['Education'])
    resume_text_skills = str(df_match_resume.iloc[[2]]['Skills & Experience'])

    result = []
    for i in range(len(df_match_job)):
        # words and sentences
        job_text_education = str(df_match_job.iloc[[i]]['Education'])
        job_text_requirements = str(df_match_job.iloc[[i]]['Key Requirements'])
    
        #sentence embeddings
        embeddings_job_education = model.encode(job_text_education)
        embeddings_resume_education = model.encode(resume_text_education)
        embeddings_job_requirements = model.encode(job_text_requirements)
        embeddings_resume_skills = model.encode(resume_text_skills)
    
        
        # sentence embeddings similarity
        education_score = util.cos_sim(embeddings_resume_education, embeddings_job_education)
        skills_score = util.cos_sim(embeddings_resume_skills, embeddings_job_requirements)
        # final combination score
        final_score = education_score.item()*0.2 + skills_score.item()
        
        temporary = list((df_match_job.iloc[[i]]['Title'], df_match_job.iloc[[i]]['FullDescription'], final_score))
        result.append(temporary)
        
    Final_result = pd.DataFrame(result, columns =['Title', 'Description', 'Final Score'])
    return Final_result

In [25]:
start = timeit.default_timer()
result = similarity_score('sentence-transformers/paraphrase-MiniLM-L3-v2', df_match_resume, df_match_job)
stop = timeit.default_timer()
print('Time: ', stop - start)  
#result
result.iloc[:int(len(df_match_job)*0.05)].sort_values(by=['Final Score'], ascending=False)

Time:  85.00568299996667


,Title,Description,Final Score
74,"74 189915 Senior Sharepoint/ Net Developer BARCELONA\nName: Title, dtype: object Name: Title, dtype: object","74 189915 Senior Sharepoint/ .Net Development Consultant BARCELONA office An international, highly technical IT Consultancy working on high profile projects and complex applications is looking for a technically strong software development consultant As a senior .Net Developer you would be technically strong and have a solid development background with indepth knowledge of .net technologies. The successful .Net Developer will have worked with .net **** with an interest in having worked on complex applications and knowledge of Sharepoint. Certifications and Technologic Formation will be provided. ROLE BASED IN BARCELONA, SPAIN. They listen and understand the needs of the market and are passionate about Microsoft Technologies. The company offers: International Mobility Annual Salary reviews Long term and adaptable Career Path depending on potential and motivation ESSENTIAL SKILLS: Be passionate about Microsoft Technologies .Net **** or **** Proven experience: .Net / Sharepoint (min.3 years) Possess ability to set and meet deadlines on multiple projects without frequent supervision Possess excellent systems analysis skills with full development lifecycle ll call you ASAP\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.517866
80,"80 232833 Business Intelligence Developer\nName: Title, dtype: object Name: Title, dtype: object","80 232833 Organisation Description An exciting opportunity to work for a new division within a global and market leading company. Job Description Engage with stakeholders to analyse reporting requirements Design metadata models and databases (both relational and dimensional) to support reporting requirements Development of Business Intelligence dashboards and reporting solutions Perform ad hoc analyses using various tools including Cognos suite and database queries Person Specification // The ideal candidate will be Technical, be able to complete the documentation and talk to senior members. 5years experience of developing data warehouse and Cognos solutions and supporting business users Experience in developing data extracts from ERP (preferably Infor and SAP) and Hyperion systems Proficient in the use SQL Server and SSIS Proficient in report development using Cognos ****x, Framework Manager, PowerPlay multidimensional cubes, security architecture and ideally Cognos TM1 Experience in developing and maintaining ETL solutions Reed Specialist Recruitment Limited is an employment agency and employment business.\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.489919
40,"40 168082 Order process Administrator\nName: Title, dtype: object Name: Title, dtype: object","40 168082 Order processor Administrator required for local company. Will be answering the telephone, direct queries to the relevant department, process orders, chase up suppliers, deal with complaints efficiently, adjust stock to fulfil orders, receive and check deliveries, match up delivery notes and invoices, obtain credit notes for damaged goods, chase payments. You must have excellent customer service skills and be accurate. This is a temp to perm position for the right person. 35 Hours per week Monday Friday. Occasional Saturdays. To apply for this position please telephone **** and ask for Tracy or Shelley.\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.467407
27,"27 45679 Senior Developer/Junior Architect (C/ASPNet) Bristol/Bath\nName: Title, dtype: object Name: Title, dtype: object","27 45679 Senior Developer/Junior Architect (C/ASP.Net) Bristol/Bath Keywords: Senior Developer, Junior Architect, C, ASP.Net, Agile. A leading organisation in the Bristol/Bath area requires a Senior C/ASP.Net Developer/Junior Architect to join its growing and successful team. The successful candidate will have strong experience of work

## -------------------------------------------------------------------

In [22]:
def similarity_score(sentence_model, df_match_resume, df_match_job):
    # sentence transformer
    model = SentenceTransformer(sentence_model)
    resume_text_education = str(df_match_resume.iloc[[2]]['Education'])
    resume_text_skills = str(df_match_resume.iloc[[2]]['Skills & Experience'])

    result = []
    for i in range(len(df_match_job)):
        # words and sentences
        job_text_education = str(df_match_job.iloc[[i]]['Education'])
        job_text_requirements = str(df_match_job.iloc[[i]]['Key Requirements'])
    
        #sentence embeddings
        embeddings_job = model.encode(job_text_education+job_text_requirements)
        embeddings_resume = model.encode(resume_text_education+resume_text_skills)
    
        
        # sentence embeddings similarity
        final_score = util.cos_sim(embeddings_job, embeddings_resume).item()
        
        temporary = list((df_match_job.iloc[[i]]['Title'], df_match_job.iloc[[i]]['FullDescription'], final_score))
        result.append(temporary)
        
    Final_result = pd.DataFrame(result, columns =['Title', 'Description', 'Final Score'])
    return Final_result

In [23]:
start = timeit.default_timer()
result = similarity_score('sentence-transformers/paraphrase-MiniLM-L3-v2', df_match_resume, df_match_job)
stop = timeit.default_timer()
print('Time: ', stop - start)  
#result
result.iloc[:int(len(df_match_job)*0.05)].sort_values(by=['Final Score'], ascending=False)

Time:  63.19440169999143


,Title,Description,Final Score
14,"14 78746 Operations Manager\nName: Title, dtype: object Name: Title, dtype: object","14 78746 This is a regional operations role to oversee a number of residential and nursing homes covering the North Hampshire and Surrey border. The role is a Monday to Friday role with oncall for emergency situations. Travel is paid between homes with excellent full benefits package offered by this national company specialising in the provision of care services to vulnerable adults and children with Autism. The role is to ensure all homes operate to CQC standards offering high levels of care. You will be involved in all aspects of the running of the homes from recruitment of staff to financial planning of new projects. If you are a confident Manager who is looking for the next step up then call Maria Smith now on (Apply online only) for a confidential chat\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.470466
80,"80 232833 Business Intelligence Developer\nName: Title, dtype: object Name: Title, dtype: object","80 232833 Organisation Description An exciting opportunity to work for a new division within a global and market leading company. Job Description Engage with stakeholders to analyse reporting requirements Design metadata models and databases (both relational and dimensional) to support reporting requirements Development of Business Intelligence dashboards and reporting solutions Perform ad hoc analyses using various tools including Cognos suite and database queries Person Specification // The ideal candidate will be Technical, be able to complete the documentation and talk to senior members. 5years experience of developing data warehouse and Cognos solutions and supporting business users Experience in developing data extracts from ERP (preferably Infor and SAP) and Hyperion systems Proficient in the use SQL Server and SSIS Proficient in report development using Cognos ****x, Framework Manager, PowerPlay multidimensional cubes, security architecture and ideally Cognos TM1 Experience in developing and maintaining ETL solutions Reed Specialist Recruitment Limited is an employment agency and employment business.\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.462369
58,"58 28461 Project Manager Jobs In Scotland\nName: Title, dtype: object Name: Title, dtype: object","58 28461 Project Manager Payments Retail Banking Financial Services Scotland A Project Manager is urgently required to work within a Financial Services Organization in Glasgow. I am interested in hearing from candidates with: ****) Proven strong Project Management skills 2) Evidence of successfully delivering business projects within a Retail Bank. ****) Experience of working with ePayments. ****) The ability to manage mixed teams of permanent members of staff, contractors and consultants. ****) The ability to demonstrate a structured approach to handling information, activities etc. ****) Excellent relationship management with a diverse range of other parties eg Business Stakeholders, IT, Industry. Rate: Up to **** Start: ASAP Location: Glasgow Duration: 12 months Project Partners is part of Hydrogen Group.\nName: FullDescription, dtype: object Name: FullDescription, dtype: object",0.457702
74,"74 189915 Senior Sharepoint/ Net Developer BARCELONA\nName: Title, dtype: object Name: Title, dtype: object","74 189915 Senior Sharepoint/ .Net Development Consultant BARCELONA office An international, highly technical IT Consultancy working on high profile projects and complex applications is looking for a technically strong software development consultant As a senior .Net Developer you would be technically strong and have a solid development background with indepth knowledge of .net technologies. The successful .Net Developer will have worked with .net **** with an interest in having worked on complex applications and knowledge of Sharepoint. Certifications and Technologic Formation will be